## Pipeline Tutorial with HeteroSecureBoost

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [73]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init    - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [74]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


### Hetero SecureBoost Example

 Before start a modeling task, data to be used should be uploaded. Please refer to this [guide](https://github.com/FederatedAI/FATE/blob/master/doc/tutorial/pipeline/pipeline_tutorial_upload.ipynb).

The `pipeline` package provides components to compose a `FATE pipeline`.

In [75]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [76]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=10000)

Define a `Reader` to load data

In [77]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

Add a `DataTransform` component to parse raw data into Data Instance

In [78]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False)

Add a `Intersection` component to perform PSI for hetero-scenario

In [79]:
intersect_0 = Intersection(name="intersect_0")

Now, we define the `HeteroSecureBoost` component. The following parameters will be set for all parties involved.

In [80]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


To show the evaluation result, an "Evaluation" component is needed.

In [81]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to pipeline, in order of execution:

    - data_transform_0 comsume reader_0's output data
    - intersect_0 comsume data_transform_0's output data
    - hetero_secureboost_0 consume intersect_0's output data
    - evaluation_0 consume hetero_secureboost_0's prediciton result on training data

Then compile our pipeline to make it ready for submission.

In [82]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile();


Now, submit(fit) our pipeline:

In [83]:
pipeline.fit()


2021-11-16 03:09:48.339 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202111160309427650180

2021-11-16 03:09:49.373 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01

2021-11-16 03:09:54.359 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_0, time elapse: 0:00:06

2021-11-16 03:10:06.253 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transform_0, time elapse: 0:00:17

2021-11-16 03:10:18.112 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component intersect_0, time elapse: 0:00:29

2021-11-16 03:12:10.125 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component hetero_secureboost_0, time elapse: 0:02:21

2021-11-16 03:12:22.960 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

Once training is done, trained model may be used for prediction. Optionally, save the trained pipeline for future use.

In [84]:
pipeline.dump("pipeline_saved.pkl");

First, deploy needed components from train pipeline

In [85]:
pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

Define new `Reader` components for reading prediction data

In [86]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

Optionally, define new `Evaluation` component.

In [87]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to predict pipeline in order of execution:

In [88]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


Then, run prediction job

In [89]:
predict_pipeline.predict()

2021-11-16 03:16:20.448 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202111160316163695630

2021-11-16 03:16:20.457 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00

2021-11-16 03:16:25.372 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component reader_1, time elapse: 0:00:04

2021-11-16 03:16:35.385 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component data_transform_0, time elapse: 0:00:14

2021-11-16 03:16:47.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component intersect_0, time elapse: 0:00:26

2021-11-16 03:17:03.030 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running component hetero_secureboost_0, time elapse: 0:00:42

2021-11-16 03:17:12.763 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:177 - Running

For more demo on using pipeline to submit jobs, please refer to [pipeline demos](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/demo)